### Installlations

In [13]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
# ✅ Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
from torchvision.transforms import v2
print("✅ Torchvision functional module is working!")


True
NVIDIA GeForce GTX 1660 SUPER
Using device: cuda
✅ Torchvision functional module is working!


# enhancement code

In [14]:
import torch
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import ipywidgets as widgets
import subprocess


## 🛠️ STEP 1: Load/Define Super-Resolution Model

In [15]:
# Cell 2: Setup dummy model (replace later with real SR model)
class UpscaleModel(torch.nn.Module):
    def __init__(self, scale_factor=2):
        super().__init__()
        self.upscale = torch.nn.Upsample(scale_factor=scale_factor, mode='bicubic', align_corners=False)

    def forward(self, x):
        return self.upscale(x)

# Instantiate and move model to GPU (if available)
model = UpscaleModel(scale_factor=2).to(device)
model.eval()
print("✅ Upscale model ready.")


✅ Upscale model ready.


## 🎥 STEP 2: Load Video, Process, and Save Upscaled Output  (Frame Processing + Muxing)

In [16]:
# Cell 3: Upscale frames and merge audio/subs

input_path = 'naruto.mkv'
temp_upscaled_path = 'temp_upscaled.mp4'
final_output_path = 'naruto_upscaled.mkv'

# Load video
cap = cv2.VideoCapture(input_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Output writer
out = cv2.VideoWriter(temp_upscaled_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width * 2, height * 2))

# Frame-wise upscale
with torch.no_grad():
    for _ in tqdm(range(frame_count), desc="Upscaling frames"):
        ret, frame = cap.read()
        if not ret:
            break
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = torch.from_numpy(img).float().permute(2, 0, 1).unsqueeze(0).to(device) / 255.0
        upscaled = model(img)
        upscaled_np = upscaled.squeeze().permute(1, 2, 0).cpu().numpy()
        upscaled_np = np.clip(upscaled_np * 255, 0, 255).astype(np.uint8)
        out_frame = cv2.cvtColor(upscaled_np, cv2.COLOR_RGB2BGR)
        out.write(out_frame)

cap.release()
out.release()
print("✅ Video upscaled and saved to:", temp_upscaled_path)


Upscaling frames:   0%|          | 0/42379 [00:00<?, ?it/s]

✅ Video upscaled and saved to: temp_upscaled.mp4


## Cell 4: Merge audio + subtitles using FFmpeg



In [17]:
import shutil

# Merge audio + subtitles
ffmpeg_cmd = [
    "ffmpeg", "-y",
    "-i", temp_upscaled_path,  # Upscaled video
    "-i", input_path,          # Original audio + subs
    "-map", "0:v:0",           # Video from new file
    "-map", "1:a?",            # All audio from original
    "-map", "1:s?",            # All subtitles from original
    "-c:v", "copy",            # Video already processed
    "-c:a", "copy",            # No re-encode
    "-c:s", "copy",            # If softsubs (e.g. .srt/.ass), else will fail
    final_output_path
]

try:
    result = subprocess.run(ffmpeg_cmd, capture_output=True, text=True, check=True)
    print("✅ Final video muxed successfully:", final_output_path)
except subprocess.CalledProcessError as e:
    print("❌ FFmpeg failed:")
    print(e.stderr)

# Optional: cleanup
if os.path.exists(temp_upscaled_path):
    os.remove(temp_upscaled_path)


✅ Final video muxed successfully: naruto_upscaled.mkv


### Requirements


In [ ]:
torch==2.2.0
opencv-python==4.9.0.80
tqdm==4.66.2
numpy==1.26.4
realesrgan==0.3.0
